<a href="https://colab.research.google.com/github/tc3oliver/LangChain-Guide/blob/dev/04_Vector_Stores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/LangChain')
with open('env_vars.txt', 'r') as file:
  for line in file:
    key, value = line.strip().split('=')
    os.environ[key] = value

!pip install langchain==0.0.247 openai

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00


In [2]:
import re
from langchain.docstore.document import Document

def split_text_into_chunks(text, source, max_length=300):
    text = text.replace('\n', '')
    text = re.sub(r'\s+', ' ', text)

    # 按照句子分割文字內容
    sentences = re.split(r'([；。！\!？\?\.])', text)
    new_sents = []
    for i in range(int(len(sentences) / 2)):
        sent = sentences[2 * i] + sentences[2 * i + 1]
        new_sents.append(sent)
    if len(sentences) % 2 == 1:
        new_sents.append(sentences[len(sentences) - 1])

    paragraphs = []
    current_length = 0
    current_paragraph = ""
    # 根據最大長度將句子分段
    for sentence in new_sents:
        sentence_length = len(sentence)
        if current_length + sentence_length <= max_length:
            current_paragraph += sentence
            current_length += sentence_length
        else:
            paragraphs.append(current_paragraph.strip())
            current_paragraph = sentence
            current_length = sentence_length
    paragraphs.append(current_paragraph.strip())

    documents = []
    metadata = {"source": source}
    # 創建 Document 物件表示每個段落
    for paragraph in paragraphs:
        new_doc = Document(page_content=paragraph, metadata=metadata)
        documents.append(new_doc)
    return documents

In [3]:
test_text = """
深度學習（英語：deep learning）是機器學習的分支，是一種以人工神經網路為架構，對資料進行表徵學習的演算法。[1][2][3][4][5] 深度學習中的形容詞「深度」是指在網路中使用多層。 早期的工作表明，線性感知器不能成為通用分類器，但具有非多項式啟用功能和一個無限寬度隱藏層的網路可以成為通用分類器。

深度學習是機器學習中一種基於對資料進行表徵學習的演算法。觀測值（例如一幅圖像）可以使用多種方式來表示，如每個像素強度值的向量，或者更抽象地表示成一系列邊、特定形狀的區域等。而使用某些特定的表示方法更容易從實例中學習任務（例如，臉部辨識或面部表情辨識[6]）。深度學習的好處是用非監督式或半監督式的特徵學習和分層特徵提取高效演算法來替代手工取得特徵。[7]

表徵學習的目標是尋求更好的表示方法並建立更好的模型來從大規模未標記資料中學習這些表示方法。表示方法來自神經科學，並鬆散地建立在類似神經系統中的資訊處理和對通信模式的理解上，如神經編碼，試圖定義拉動神經元的反應之間的關係以及大腦中的神經元的電活動之間的關係。[8]

至今已有數種深度學習框架，如深度神經網路、卷積神經網路和深度置信網路和迴圈神經網路已被應用在電腦視覺、語音辨識、自然語言處理、音訊辨識與生物資訊學等領域並取得了極好的效果。

另外，「深度學習」已成為時髦術語，或者說是人工神經網路的品牌重塑。[9][10]

簡介
深度學習框架，尤其是基於人工神經網路的框架可以追溯到1980年福島邦彥提出的新認知機[11]，而人工神經網路的歷史更為久遠。1989年，揚·勒丘恩（Yann LeCun）等人開始將1974年提出的標準反向傳播演算法[12]應用於深度神經網路，這一網路被用於手寫郵政編碼辨識。儘管演算法可以成功執行，但計算代價非常巨大，神經網路的訓練時間達到了3天，因而無法投入實際使用[13]。許多因素導致了這一緩慢的訓練過程，其中一種是由于爾根·施密德胡伯的學生賽普·霍克賴特於1991年提出的梯度消失問題[14][15]。

最早的進行一般自然雜亂圖像中自然物體辨識的深度學習網路是翁巨揚（Juyang Weng）等在1991和1992發表的生長網（Cresceptron）[16][17][18]。它也是第一個提出了後來很多實驗廣泛採用的一個方法：現在稱為最大匯集（max-pooling）以用於處理大物體的變形等問題。生長網不僅直接從雜亂自然場景中學習老師指定的一般物體，還用網路反向分析的方法把圖像內被辨識了的物體從背景圖像中分割出來。

2007年前後，傑弗里·辛頓和魯斯蘭·薩拉赫丁諾夫（Ruslan Salakhutdinov）提出了一種在前饋神經網路中進行有效訓練的演算法。這一演算法將網路中的每一層視為無監督的受限玻爾茲曼機，再使用有監督的反向傳播演算法進行調優[19]。在此之前的1992年，在更為普遍的情形下，施密德胡伯也曾在迴圈神經網路上提出一種類似的訓練方法，並在實驗中證明這一訓練方法能夠有效提高有監督學習的執行速度[20][21].

自深度學習出現以來，它已成為很多領域，尤其是在電腦視覺和語音辨識中，成為各種領先系統的一部分。在通用的用於檢驗的資料集，例如語音辨識中的TIMIT和圖像辨識中的ImageNet, Cifar10上的實驗證明，深度學習能夠提高辨識的精度。與此同時，神經網路也受到了其他更加簡單歸類模型的挑戰，支持向量機等模型在20世紀90年代到21世紀初成為過流行的機器學習演算法。

硬體的進步也是深度學習重新獲得關注的重要因素。高效能圖形處理器的出現極大地提高了數值和矩陣運算的速度，使得機器學習演算法的執行時間得到了顯著的縮短[22][23]。

由於腦科學方面的大量研究已表明人腦網路不是一個級聯的結構，深度學習網路在2001年後正逐漸被更有潛力的基於腦模型的網路[24][25]所替代。


基本概念
主條目：層 (深度學習)
深度學習的基礎是機器學習中的分散表示（distributed representation）。分散表示假定觀測值是由不同因子相互作用生成。在此基礎上，深度學習進一步假定這一相互作用的過程可分為多個層次，代表對觀測值的多層抽象。不同的層數和層的規模可用於不同程度的抽象[3]。

深度學習運用了這分層次抽象的思想，更高層次的概念從低層次的概念學習得到。這一分層結構常常使用貪婪演算法逐層構建而成，並從中選取有助於機器學習的更有效的特徵[3]。

不少深度學習演算法都以無監督學習的形式出現，因而這些演算法能被應用於其他演算法無法企及的無標籤資料，這一類資料比有標籤資料更豐富，也更容易獲得。這一點也為深度學習贏得了重要的優勢[3]。

人工神經網路下的深度學習
一部分最成功的深度學習方法涉及到對人工神經網路的運用。人工神經網路受到了1959年由諾貝爾獎得主大衛·休伯爾（David H. Hubel）和托斯坦·威澤爾（Torsten Wiesel）提出的理論啟發。休伯爾和威澤爾發現，在大腦的初級視覺皮層中存在兩種細胞：簡單細胞和複雜細胞，這兩種細胞承擔不同層次的視覺感知功能。受此啟發，許多神經網路模型也被設計為不同節點之間的分層模型[26]。

福島邦彥提出的新認知機引入了使用無監督學習訓練的卷積神經網路。揚·勒丘恩將有監督的反向傳播演算法應用於這一架構[27]。事實上，從反向傳播演算法自20世紀70年代提出以來，不少研究者都曾試圖將其應用於訓練有監督的深度神經網路，但最初的嘗試大都失敗。賽普·霍克賴特在其博士論文中將失敗的原因歸結為梯度消失，這一現象同時在深度前饋神經網路和迴圈神經網路中出現，後者的訓練過程類似深度網路。在分層訓練的過程中，本應用於修正模型參數的誤差隨著層數的增加指數遞減，這導致了模型訓練的效率低下[28][29]。

為了解決這一問題，研究者們提出了一些不同的方法。于爾根·施密德胡伯於1992年提出多層級網路，利用無監督學習訓練深度神經網路的每一層，再使用反向傳播演算法進行調優。在這一模型中，神經網路中的每一層都代表觀測變數的一種壓縮表示，這一表示也被傳遞到下一層網路[20]。

另一種方法是賽普·霍克賴特和于爾根·施密德胡伯提出的長短期記憶神經網路（LSTM）[30]。2009年，在ICDAR 2009舉辦的連筆手寫辨識競賽中，在沒有任何先驗知識的情況下，深度多維長短期記憶神經網路取得了其中三場比賽的勝利[31][32]。

斯文·貝克提出了在訓練時只依賴梯度符號的神經抽象金字塔模型，用以解決圖像重建和人臉定位的問題[33]。

其他方法同樣採用了無監督預訓練來構建神經網路，用以發現有效的特徵，此後再採用有監督的反向傳播以區分有標籤資料。傑弗里·辛頓等人於2006年提出的深度模型提出了使用多層隱變數學習高層表示的方法。這一方法使用斯摩棱斯基於1986年提出的受限玻爾茲曼機[34]對每一個包含高層特徵的層進行建模。模型保證了資料的對數似然下界隨著層數的提升而遞增。當足夠多的層數被學習完畢，這一深層結構成為一個生成模型，可以通過自上而下的採樣重構整個資料集[35]。辛頓聲稱這一模型在高維結構化資料上能夠有效地提取特徵[36]。

吳恩達和傑夫·迪恩領導的Google大腦團隊建立了一個僅通過YouTube影片學習高層概念（例如貓）的神經網路[37] [38]。

其他方法依賴了現代電腦的強大計算能力，尤其是GPU。2010年，在于爾根·施密德胡伯位於瑞士人工智慧實驗室IDSIA的研究組中，丹·奇雷尚（Dan Ciresan）和他的同事展示了利用GPU直接執行反向傳播演算法而忽視梯度消失問題的存在。這一方法在揚·勒丘恩等人給出的手寫辨識MNIST資料集上戰勝了已有的其他方法[22]。

截止2011年，前饋神經網路深度學習中最新的方法是交替使用卷積層（convolutional layers）和最大值池化層（max-pooling layers）並加入單純的分類層作為頂端。訓練過程也無需引入無監督的預訓練[39][40]。從2011年起，這一方法的GPU實現[39]多次贏得了各類圖型識別競賽的勝利，包括IJCNN 2011交通標誌辨識競賽[41]和其他比賽。

這些深度學習演算法也是最先在某些辨識任務上達到和人類表現具備同等競爭力的演算法[42]。

深度學習結構
通常將具有兩層或兩層以上隱藏層的神經網路叫做深度神經網路。與淺層神經網路類似，深度神經網路也能夠為複雜非線性系統提供建模，但多出的層次為模型提供了更高的抽象層次，因而提高了模型的能力。深度神經網路通常都是前饋神經網路，但也有語言建模等方面的研究將其拓展到迴圈神經網路[43]。卷積深度神經網路（Convolutional Neural Networks, CNN）在電腦視覺領域得到了成功的應用[44]。此後，卷積神經網路也作為聽覺模型被使用在自動語音辨識領域，較以往的方法獲得了更優的結果[45]。

深度神經網路
深度神經網路（Deep Neural Networks, DNN）是一種判別模型，可以使用反向傳播演算法進行訓練。權重更新可以使用下式進行隨機梯度下降法求解：

Δ

深度神經網路的問題
與其他神經網路模型類似，如果僅僅是簡單地訓練，深度神經網路可能會存在很多問題。常見的兩類問題是過擬合和過長的運算時間。

深度神經網路很容易產生過擬合現象，因為增加的抽象層使得模型能夠對訓練資料中較為罕見的依賴關係進行建模。對此，權重遞減（
ℓ
2
 \ell_2 正規化）或者稀疏（
ℓ
1
 \ell_1 -正規化）等方法可以利用在訓練過程中以減小過擬合現象[47]。另一種較晚用於深度神經網路訓練的正規化方法是丟棄法（"dropout" regularization），即在訓練中隨機丟棄一部分隱層單元來避免對較為罕見的依賴進行建模[48]。目前比較廣泛使用的是批歸一化(Batch Normalization,BN) , 其本質上是在訓練過程加入噪音 , 從而讓模型得到更好的魯棒性 , 其特性令超深神經網路可以更好的訓練。

反向傳播演算法和梯度下降法由於其實現簡單，與其他方法相比能夠收斂到更好的局部最佳值而成為神經網路訓練的通行方法。但是，這些方法的計算代價很高，尤其是在訓練深度神經網路時，因為深度神經網路的規模（即層數和每層的節點數）、學習率、初始權重等眾多參數都需要考慮。掃描所有參數由於時間代價的原因並不可行，因而小批次訓練（mini-batching），即將多個訓練樣本組合進行訓練而不是每次只使用一個樣本進行訓練，被用於加速模型訓練[49]。而最顯著地速度提升來自GPU，因為矩陣和向量計算非常適合使用GPU實現。但使用大規模叢集進行深度神經網路訓練仍然存在困難，因而深度神經網路在訓練並列化方面仍有提升的空間。

深度置信網路

一個包含完全連接可見層和隱層的受限玻爾茲曼機（RBM）。注意到可見層單元和隱層單元內部彼此不相連。
深度置信網路（deep belief networks，DBN）是一種包含多層隱單元的概率生成模型，可被視為多層簡單學習模型組合而成的複合模型[50]。

深度置信網路可以作為深度神經網路的預訓練部分，並為網路提供初始權重，再使用反向傳播或者其它判定演算法作為調優的手段。這在訓練資料較為缺乏時很有價值，因為不恰當的初始化權重會顯著影響最終模型的效能，而預訓練獲得的權重在權值空間中比隨機權重更接近最佳的權重。這不僅提升了模型的效能，也加快了調優階段的收斂速度[51]。

深度置信網路中的每一層都是典型的受限玻爾茲曼機（restricted Boltzmann machine，RBM），可以使用高效的無監督逐層訓練方法進行訓練。受限玻爾茲曼機是一種無向的基於能量的生成模型，包含一個輸入層和一個隱層。圖中對的邊僅在輸入層和隱層之間存在，而輸入層節點內部和隱層節點內部則不存在邊。單層RBM的訓練方法最初由傑弗里·辛頓在訓練「專家乘積」中提出，被稱為對比分歧（contrast divergence, CD）。對比分歧提供了一種對最大似然的近似，被理想地用於學習受限玻爾茲曼機的權重[49]。當單層RBM被訓練完畢後，另一層RBM可被堆疊在已經訓練完成的RBM上，形成一個多層模型。每次堆疊時，原有的多層網路輸入層被初始化為訓練樣本，權重為先前訓練得到的權重，該網路的輸出作為新增RBM的輸入，新的RBM重複先前的單層訓練過程，整個過程可以持續進行，直到達到某個期望中的終止條件[2]。

儘管對比分歧對最大似然的近似十分粗略（對比分歧並不在任何函數的梯度方向上），但經驗結果證實該方法是訓練深度結構的一種有效的方法[49]。

卷積神經網路
主條目：卷積神經網路
卷積神經網路（convolutional neural networks，CNN）由一個或多個卷積層和頂端的全連通層（對應經典的神經網路）組成，同時也包括關聯權重和池化層（pooling layer）。這一結構使得卷積神經網路能夠利用輸入資料的二維結構。與其他深度學習結構相比，卷積神經網路在圖像和語音辨識方面能夠給出更優的結果。這一模型也可以使用反向傳播演算法進行訓練。相比較其他深度、前饋神經網路，卷積神經網路需要估計的參數更少，使之成為一種頗具吸引力的深度學習結構[52]。

卷積深度置信網路
卷積深度置信網路（convolutional deep belief networks，CDBN）是深度學習領域較新的分支。在結構上，卷積深度置信網路與卷積神經網路在結構上相似。因此，與卷積神經網路類似，卷積深度置信網路也具備利用圖像二維結構的能力，與此同時，卷積深度信念網路也擁有深度置信網路的預訓練優勢。卷積深度置信網路提供了一種能被用於訊號和圖像處理任務的通用結構，也能夠使用類似深度置信網路的訓練方法進行訓練[53]。

結果
語音辨識
下表中的結果展示了深度學習在通行的TIMIT資料集上的結果。TIMIT包含630人的語音資料，這些人持八種常見的美式英語口音，每人閱讀10句話。這一資料在深度學習發展之初常被用於驗證深度學習結構[54]。TIMIT資料集較小，使得研究者可以在其上實驗不同的模型組態。

方法	聲音誤差率 (PER, %)
隨機初始化RNN	26.1
貝葉斯三音子GMM-HMM	25.6
單音子重複初始化DNN	23.4
單音子DBN-DNN	22.4
帶BMMI訓練的三音子GMM-HMM	21.7
共享池上的單音子DBN-DNN	20.7
卷積DNN	20.0
圖像分類
圖像分類領域中一個公認的評判資料集是MNIST資料集。MNIST由手寫阿拉伯數字組成，包含60,000個訓練樣本和10,000個測試樣本。與TIMIT類似，它的資料規模較小，因而能夠很容易地在不同的模型組態下測試。Yann LeCun的網站給出了多種方法得到的實驗結果[55]。截至2012年，最好的判別結果由Ciresan等人在當年給出，這一結果的錯誤率達到了0.23%[56]。

深度學習與神經科學
電腦領域中的深度學習與20世紀90年代由認知神經科學研究者提出的大腦發育理論（尤其是皮層發育理論）密切相關[57]。對這一理論最容易理解的是傑弗里·艾爾曼於1996年出版的專著《對天賦的再思考》（Rethinking Innateness）[58]（參見斯拉格和詹森[59]以及奎茲和賽傑諾維斯基[60]的表述）。由於這些理論給出了實際的神經計算模型，因而它們是純計算驅動的深度學習模型的技術先驅。這些理論指出，大腦中的神經元組成了不同的層次，這些層次相互連接，形成一個過濾體系。在這些層次中，每層神經元在其所處的環境中取得一部分資訊，經過處理後向更深的層級傳遞。這與後來的單純與計算相關的深度神經網路模型相似。這一過程的結果是一個與環境相協調的自組織的堆疊式的轉換器。正如1995年在《紐約時報》上刊登的那樣，「……嬰兒的大腦似乎受到所謂『營養因素』的影響而進行著自我組織……大腦的不同區域依次相連，不同層次的腦組織依照一定的先後順序發育成熟，直至整個大腦發育成熟。」[61]

深度結構在人類認知演化和發展中的重要性也在認知神經學家的關注之中。發育時間的改變被認為是人類和其他靈長類動物之間智力發展差異的一個方面[62]。在靈長類中，人類的大腦在出生後的很長時間都具備可塑性，但其他靈長類動物的大腦則在出生時就幾乎完全定型。因而，人類在大腦發育最具可塑性的階段能夠接觸到更加複雜的外部場景，這可能幫助人類的大腦進行調節以適應快速變化的環境，而不是像其他動物的大腦那樣更多地受到遺傳結構的限制。這樣的發育時間差異也在大腦皮層的發育時間和大腦早期自組織中從刺激環境中取得資訊的改變得到體現。當然，伴隨著這一可塑性的是更長的兒童期，在此期間人需要依靠撫養者和社會群體的支援和訓練。因而這一理論也揭示了人類演化中文化和意識共同進化的現象[63]。

公眾視野中的深度學習
深度學習常常被看作是通向真正人工智慧的重要一步[64]，因而許多機構對深度學習的實際應用抱有濃厚的興趣。2013年12月，Facebook宣布雇用楊立昆為其新建的人工智慧實驗室的主管，這一實驗室將在加州、倫敦和紐約設立分支機構，幫助Facebook研究利用深度學習演算法進行類似自動標記相片中使用者姓名這樣的任務[65]。

2013年3月，傑弗里·辛頓和他的兩位研究生亞歷克斯·克里澤夫斯基和伊利婭·蘇特斯科娃被Google公司雇用，以提升現有的機器學習產品並協助處理Google日益增長的資料。Google同時併購了辛頓創辦的公司DNNresearch[66]。

2016年3月，以深度學習開發的圍棋程式AlphaGo首度在比賽中擊敗人類頂尖選手，形成廣泛的討論。

批評
對深度學習的主要批評是許多方法缺乏理論支撐。大多數深度結構僅僅是梯度下降的某些變式。儘管梯度下降法已經被充分地研究，但理論涉及的其他演算法，例如對比分歧演算法，並沒有獲得充分的研究，其收斂性等問題仍不明確。深度學習方法常常被視為黑盒，大多數的結論確認都由經驗而非理論來確定。

也有學者認為，深度學習應當被視為通向真正人工智慧的一條途徑，而不是一種包羅萬象的解決方案。儘管深度學習的能力很強，但和真正的人工智慧相比，仍然缺乏諸多重要的能力。理論心理學家加里·馬庫斯指出：

就現實而言，深度學習只是建造智慧型機器這一更大挑戰中的一部分。這些技術缺乏表達因果關係的手段……缺乏進行邏輯推理的方法，而且遠沒有具備整合抽象知識，例如物品屬性、代表和典型用途的資訊。最為強大的人工智慧系統，例如IBM的人工智慧系統華生，僅僅把深度學習作為一個包含從貝葉斯推理和演繹推理等技術的複雜技術集合中的組成部分[67]。
"""

In [4]:
class TextLoader:
    def __init__(self, text, source):
        self.text = text
        self.source = source

    def load(self, max_length=300):
        documents = split_text_into_chunks(self.text, self.source, max_length)
        return documents


In [6]:
docs = TextLoader(test_text, 'wiki').load()

In [5]:
!pip install chromadb mmh3 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to b

In [7]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import mmh3

def murmurhash(text):
    hash_value = mmh3.hash(text)
    return str(hash_value)

# 獲取當前工作目錄
ABS_PATH = os.getcwd()

# 組合 DB_DIR
DB_DIR = os.path.join(ABS_PATH, "db")

def persist_embedding(documents, collection_name):
    ids = [murmurhash(d.page_content) for d in documents]

    client = chromadb.PersistentClient(path=DB_DIR)

    embedding = OpenAIEmbeddings()

    vectorstore = Chroma(
        collection_name=collection_name,
        client=client,
        embedding_function=embedding,
        persist_directory=DB_DIR)

    vectorstore.add_documents(documents=documents, embedding=embedding, ids=ids)
    vectorstore.persist()

In [8]:
persist_embedding(docs, 'wiki')

In [9]:
client = chromadb.PersistentClient(path=DB_DIR)
embedding = OpenAIEmbeddings()

vectorstore = Chroma(
        collection_name='wiki',
        client=client,
        embedding_function=embedding,
        persist_directory=DB_DIR)

In [10]:
query = "什麼是深度學習"

docs = vectorstore.similarity_search(query)

In [11]:
docs[0]

Document(page_content='深度學習（英語：deep learning）是機器學習的分支，是一種以人工神經網路為架構，對資料進行表徵學習的演算法。[1][2][3][4][5] 深度學習中的形容詞「深度」是指在網路中使用多層。 早期的工作表明，線性感知器不能成為通用分類器，但具有非多項式啟用功能和一個無限寬度隱藏層的網路可以成為通用分類器。深度學習是機器學習中一種基於對資料進行表徵學習的演算法。觀測值（例如一幅圖像）可以使用多種方式來表示，如每個像素強度值的向量，或者更抽象地表示成一系列邊、特定形狀的區域等。而使用某些特定的表示方法更容易從實例中學習任務（例如，臉部辨識或面部表情辨識[6]）。', metadata={'source': 'wiki'})

In [17]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES").
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

Always return the "source" section in your answers. The source comes from the metadata of Context
These are historical data, so please ignore the terms like 'this week,' 'this month,' 'today,' etc. Please reply using the language that the user is using, and pay special attention to the differences between Traditional Chinese and Simplified Chinese.
===

Context: {summaries}

===

question: {question}

===

answer:
"""

def create_prompt_with_sources():
    return PromptTemplate(
        template=prompt_template, input_variables=["summaries", "question"]
    )

In [18]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
prompt = create_prompt_with_sources()
chain = load_qa_with_sources_chain(llm=llm, chain_type="stuff", prompt=prompt)

In [19]:
retriever = vectorstore.as_retriever()
docs = retriever.get_relevant_documents(query)
result = chain({"input_documents": docs, "question": query}, return_only_outputs=False)

In [20]:
result

{'input_documents': [Document(page_content='深度學習（英語：deep learning）是機器學習的分支，是一種以人工神經網路為架構，對資料進行表徵學習的演算法。[1][2][3][4][5] 深度學習中的形容詞「深度」是指在網路中使用多層。 早期的工作表明，線性感知器不能成為通用分類器，但具有非多項式啟用功能和一個無限寬度隱藏層的網路可以成為通用分類器。深度學習是機器學習中一種基於對資料進行表徵學習的演算法。觀測值（例如一幅圖像）可以使用多種方式來表示，如每個像素強度值的向量，或者更抽象地表示成一系列邊、特定形狀的區域等。而使用某些特定的表示方法更容易從實例中學習任務（例如，臉部辨識或面部表情辨識[6]）。', metadata={'source': 'wiki'}),
  Document(page_content='深度學習的好處是用非監督式或半監督式的特徵學習和分層特徵提取高效演算法來替代手工取得特徵。[7]表徵學習的目標是尋求更好的表示方法並建立更好的模型來從大規模未標記資料中學習這些表示方法。表示方法來自神經科學，並鬆散地建立在類似神經系統中的資訊處理和對通信模式的理解上，如神經編碼，試圖定義拉動神經元的反應之間的關係以及大腦中的神經元的電活動之間的關係。[8]至今已有數種深度學習框架，如深度神經網路、卷積神經網路和深度置信網路和迴圈神經網路已被應用在電腦視覺、語音辨識、自然語言處理、音訊辨識與生物資訊學等領域並取得了極好的效果。另外，「深度學習」已成為時髦術語，或者說是人工神經網路的品牌重塑。', metadata={'source': 'wiki'}),
  Document(page_content='批評對深度學習的主要批評是許多方法缺乏理論支撐。大多數深度結構僅僅是梯度下降的某些變式。儘管梯度下降法已經被充分地研究，但理論涉及的其他演算法，例如對比分歧演算法，並沒有獲得充分的研究，其收斂性等問題仍不明確。深度學習方法常常被視為黑盒，大多數的結論確認都由經驗而非理論來確定。也有學者認為，深度學習應當被視為通向真正人工智慧的一條途徑，而不是一種包羅萬象的解決方案。儘管深度學習的能力很強，但和真正的人工智慧相比，仍然缺乏諸多重要的能力。理論心理學家加里·馬庫斯指出：就現實而言，深度學習只是建造智慧型機器這一

In [21]:
result['question']

'什麼是深度學習'

In [22]:
result['output_text']

'深度學習是機器學習的一種分支，它是一種以人工神經網路為架構，對資料進行表徵學習的演算法。深度學習使用多層的網路來進行學習，並且可以用非監督式或半監督式的特徵學習和分層特徵提取來替代手工取得特徵。它的目標是尋求更好的表示方法並建立更好的模型來從大規模未標記資料中學習這些表示方法。深度學習已被應用在電腦視覺、語音辨識、自然語言處理、音訊辨識與生物資訊學等領域並取得了極好的效果。然而，深度學習也存在一些批評，例如缺乏理論支撐和被視為黑盒等。深度學習被認為是通向真正人工智慧的一條途徑，但仍然缺乏諸多重要的能力。[1][2][3][4][5][6][7][8][19][20][21]\nSources: \n[1] https://zh.wikipedia.org/wiki/%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92\n[2] https://zh.wikipedia.org/wiki/%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92#%E5%8E%9F%E7%90%86\n[3] https://zh.wikipedia.org/wiki/%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92#%E6%87%89%E7%94%A8\n[4] https://zh.wikipedia.org/wiki/%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92#%E6%89%B9%E8%A9%95\n[5] https://zh.wikipedia.org/wiki/%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92#%E5%8F%83%E8%80%83%E6%96%87%E7%8D%BB'